<a href="https://colab.research.google.com/github/rrfsantos/Large-Language-Model-Agents-MOOC-Fall-2024/blob/main/AI%20Agentic%20Design%20Patterns%20with%20AutoGen/L6_Planning_and_Stock_Report_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 6: Planning and Stock Report Generation

## Setup

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#!pip install -r /content/drive/MyDrive/AutoGen/requirements.txt

In [8]:
from typing_extensions import Annotated
from dotenv import load_dotenv
import warnings
import os
from autogen import ConversableAgent, AssistantAgent, initiate_chats, register_function
from autogen.coding import LocalCommandLineCodeExecutor
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [9]:
# Load environment variables from .env file
load_dotenv("/content/drive/MyDrive/AutoGen/env")
llm_config = {"config_list": [{"model": "gpt-4o", "api_key" : os.getenv("OPENAI_API_KEY")}]}

## The task!

In [10]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [11]:
import autogen

In [12]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

[autogen.oai.client: 10-28 19:43:29] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [13]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

[autogen.oai.client: 10-28 19:43:29] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [14]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

[autogen.oai.client: 10-28 19:43:29] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


**Note**: In this lesson, you'll use an alternative method of code execution by providing a dict config. However, you can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [15]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [16]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

[autogen.oai.client: 10-28 19:43:29] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Define the group chat

In [17]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [18]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


[autogen.oai.client: 10-28 19:43:29] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Start the group chat!

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

In [19]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about Nvidia's stock price performance over the past month, we need to gather specific information using Python. Here are the steps and data required:

1. **Retrieve Stock Price Data:**
   - Obtain historical stock price data for Nvidia (NVDA) over the past month (from March 23, 2024, to April 23, 2024).

   *Python Libraries and Methods to Use*:
   - Use the `yfinance` library to pull historical stock data. This library provides easy access to Yahoo Finance data.

   ```python
   import yfinance as yf
   nvda_data = yf.download("NVDA", start="2024-03-23", end="2024-04-23")
   ```

2. **Analyze Stock Price Trends:**
   - Calculate key statistics such as the highest and lowest stock price, average

## Add a speaker selection policy

In [20]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

[autogen.oai.client: 10-28 19:44:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-28 19:44:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-28 19:44:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-28 19:44:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [21]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [22]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

[autogen.oai.client: 10-28 19:44:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about Nvidia's stock price performance over the past month, we need to gather specific information using Python. Here are the steps and data required:

1. **Retrieve Stock Price Data:**
   - Obtain historical stock price data for Nvidia (NVDA) over the past month (from March 23, 2024, to April 23, 2024).

   *Python Libraries and Methods to Use*:
   - Use the `yfinance` library to pull historical stock data. This library provides easy access to Yahoo Finance data.

   ```python
   import yfinance as yf
   nvda_data = yf.download("NVDA", start="2024-03-23", end="2024-04-23")
   ```

2. **Analyze Stock Price Trends:**
   - Calculate key statistics such as the highest and lowest stock price, average

**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.